In [2]:
import pandas as pd
import numpy as np


In [30]:
averaged_car_measurements = pd.read_csv('data/averaged_car_measurements.csv')

#Change date to datetime, format is %Y-%m-%d
averaged_car_measurements['date'] = pd.to_datetime(averaged_car_measurements['date'], format='%Y-%m-%d')

averaged_car_measurements.head()

,Hour,date,NO_ugm3,NO2_ugm3,O3_ugm3,CO_mgm3,CO2_mgm3,PM25_ugm3,SiteID,Lat,Long,month,day_of_week
0,8,2021-06-28,12.044255,0.085176,39.303789,0.351863,878.897362,5.072396,1,53.348754,-6.257607,6,0
1,9,2021-06-28,5.263025,9.600681,29.413390,0.350409,842.139168,7.998587,1,53.348754,-6.257607,6,0
2,9,2021-08-16,10.318074,3.006013,61.380364,0.380456,809.073556,6.122852,1,53.348754,-6.257607,8,0
3,9,2021-10-22,456.213318,NaN,NaN,0.432339,850.479574,5.752877,1,53.348754,-6.257607,10,4
4,9,2022-07-29,15.093107,NaN,NaN,0.415959,805.524520,4.757151,1,53.348754,-6.257607,7,4


In [24]:
#Prep hourly_weather data
hourly_weather = pd.read_csv('Weather-Air_pollution/merged_weather_data.csv')
#Convert hour to integer
hourly_weather['hour'] = hourly_weather['hour'].str.replace(':00', '')
hourly_weather['hour'] = hourly_weather['hour'].astype(int)

#Drop latitute, longitude and datetime columns as they are not needed
hourly_weather = hourly_weather.drop(['latitude', 'longitude','datetime'], axis=1)

#Change date to datetime, format is %d/%m/%Y
hourly_weather['date'] = pd.to_datetime(hourly_weather['date'], format='%d/%m/%Y')

hourly_weather.head()

,date,hour,avgtempC,maxtempC,mintempC,sunHour,uvIndex,humidity,winddirDegree,windspeedKmph,cloudcover,precipMM,pressure
0,2021-05-01,0,6,8,3,13,3,80,305,9,5,0.0,1017
1,2021-05-01,1,6,8,3,13,3,82,306,10,15,0.0,1017
2,2021-05-01,2,6,8,3,13,3,83,306,11,25,0.0,1017
3,2021-05-01,3,6,8,3,13,3,85,307,12,35,0.0,1017
4,2021-05-01,4,6,8,3,13,3,84,308,12,41,0.0,1017


In [47]:
#Prepare Air pollution sensor data
air_pollution_sensors = pd.read_csv('Weather-Air_pollution/Air_Pollution_Sensors.csv')

#Extract hour and date from datetime column
air_pollution_sensors['hour'] = pd.to_datetime(air_pollution_sensors['datetime']).dt.hour
air_pollution_sensors['date'] = pd.to_datetime(air_pollution_sensors['datetime']).dt.date
#Drop datetime column
air_pollution_sensors = air_pollution_sensors.drop(['datetime'], axis=1)

#Show the levels of the column 'serial_number'
#print(air_pollution_sensors['serial_number'].value_counts())

#Pivot the data for each sensor, indicated by the column 'serial_number'
air_pollution_sensors2 = air_pollution_sensors.pivot(index=['date', 'hour'], columns='serial_number')

air_pollution_sensors2.shape



(5943, 324)

In [48]:
30*24*20


14400

In [36]:
## Weather data
#Left join the two dataframes on hour and date together
Full_data = pd.merge(averaged_car_measurements, hourly_weather, how='left', left_on=['date','Hour'], right_on=['date', 'hour'])
#Drop hour column
Full_data = Full_data.drop(['Hour'], axis=1)


Full_data.head()

,date,NO_ugm3,NO2_ugm3,O3_ugm3,CO_mgm3,CO2_mgm3,PM25_ugm3,SiteID,Lat,Long,...,maxtempC,mintempC,sunHour,uvIndex,humidity,winddirDegree,windspeedKmph,cloudcover,precipMM,pressure
0,2021-06-28,12.044255,0.085176,39.303789,0.351863,878.897362,5.072396,1,53.348754,-6.257607,...,17,10,16,4,75,114,11,21,0.0,1018
1,2021-06-28,5.263025,9.600681,29.413390,0.350409,842.139168,7.998587,1,53.348754,-6.257607,...,17,10,16,4,72,0,12,18,0.0,1018
2,2021-08-16,10.318074,3.006013,61.380364,0.380456,809.073556,6.122852,1,53.348754,-6.257607,...,19,12,13,4,73,284,23,17,0.0,1020
3,2021-10-22,456.213318,NaN,NaN,0.432339,850.479574,5.752877,1,53.348754,-6.257607,...,12,8,3,3,70,273,22,75,0.0,1022
4,2022-07-29,15.093107,NaN,NaN,0.415959,805.524520,4.757151,1,53.348754,-6.257607,...,21,14,12,5,63,126,5,19,0.0,1017
